# Finalize LA high schools

Prepare the final shapefile of LA high schools for geospatial analysis.

In [1]:
import os
import collections
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [26]:
import warnings
warnings.simplefilter("ignore")

In [3]:
%store -r

Read in our list of LA high schools that have not moved in the last five years.

In [8]:
schools = pd.read_csv(os.path.join(output_dir, "unmoved-la-highschools.csv"))

In [9]:
len(schools)

329

Some campuses are home to multiple high schools. For the purposes of this analysis, we only want to count them once. To accomplish that, will drop the schools that appear more than once. In cases where a charter and a traditional school share a campus, we will keep the traditional school.

In [10]:
campuses = schools.sort_values('Charter').drop_duplicates(['Street'], keep='first')

In [11]:
campuses.CDSCode.nunique()

295

Clean up the column headers

In [12]:
clean_columns = collections.OrderedDict((
    ('CDSCode', 'cds_code'),
    ('District', 'district'),
    ('School', 'name'),
    ('Street', 'street'),
    ('City', 'city'),
    ('Zip', 'zipcode'),
    ('Charter', 'charter'),
    ('Magnet', 'magnet'),
    ('OpenDate', 'open_date'),
    ('Longitude', 'x'),
    ('Latitude', 'y'),
))

In [14]:
trimmed_campuses = campuses[list(clean_columns.keys())].rename(columns=clean_columns)

Convert to GeoDataFrame

In [16]:
trimmed_campuses[['x', 'y']] = trimmed_campuses[['x', 'y']].apply(pd.to_numeric)

In [17]:
trimmed_campuses['geometry'] = trimmed_campuses.apply(lambda z: Point(z.x, z.y), axis=1)

In [18]:
campuses_gdf = gpd.GeoDataFrame(trimmed_campuses, crs={'init': 'epsg:4269'})

Reproject into feet so we can draw a mile buffer from the point

In [19]:
campuses_gdf_feet = campuses_gdf.to_crs({'init':'epsg:2229'})

In [20]:
campuses_gdf_feet.crs

{'init': 'epsg:2229'}

Draw a buffer

In [21]:
campuses_gdf_buffer = campuses_gdf_feet.copy()

In [22]:
campuses_gdf_buffer['geometry'] = campuses_gdf_feet.geometry.buffer(5280)

Make sure the CDS code stays a string

In [23]:
campuses_gdf_buffer.cds_code = campuses_gdf_buffer.cds_code.astype(str)

Write out a shapefile

In [27]:
campuses_gdf_buffer.to_file(os.path.join(output_dir, "school-buffers.shp"))